In [1]:
# -*- coding: utf-8 -*-
import pandas as pd

In [2]:
df = pd.read_csv('../data/train/SQL注入.csv')
df.fillna('__NaN__', inplace=True)
df = df[:10]
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          10 non-null     int64 
 1   method      10 non-null     object
 2   user_agent  10 non-null     object
 3   url         10 non-null     object
 4   refer       10 non-null     object
 5   body        10 non-null     object
 6   lable       10 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 688.0+ bytes


,id,method,user_agent,url,refer,body,lable
0,17902,GET,Dalvik/2.1.0 (Linux; U; Android 11; SM-G9860 B...,/livemsg?ad_type=WL_WK&ty=web&pu=0&openudid=ed...,__NaN__,GET /livemsg?ad_type=WL_WK&ty=web&pu=0&openudi...,1
1,190,GET,Dalvik/2.1.0 (Linux; U; Android 11; Mi 10 Buil...,/livemsg?ad_type=WL_WK&ty=web&pu=0&openudid=d6...,__NaN__,GET /livemsg?ad_type=WL_WK&ty=web&pu=0&openudi...,1
2,8799,GET,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,/(select%20extractvalue(xmltype('%3c%3fxml%20v...,__NaN__,GET /(select%20extractvalue(xmltype('%3c%3fxml...,1
3,8788,GET,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,/ftp/quarantine/?(select%20load_file('%5c%5c%5...,__NaN__,GET /ftp/quarantine/?(select%20load_file('%5c%...,1
4,16030,GET,Dalvik/2.1.0 (Linux; U; Android 9; MI 9 SE MIU...,/livemsg?ad_type=WL_WK&oadid=&ty=web&pu=0&adap...,__NaN__,GET /livemsg?ad_type=WL_WK&oadid=&ty=web&pu=0&...,1


In [3]:
x = df['body'][0]
x

'GET /livemsg?ad_type=WL_WK&ty=web&pu=0&openudid=ed7c82fd9fe8cd29fcab434ece87b269&vid=d3028h8c7mg&duration=49&offline=0&adselector=Y&from=5&live=0&channelId=news_news_top&oadid=&articleId=20191130V0F0J200&adaptor=1&appversion=211115&l=116&newnettype=1&chid=2&isAdVideo=false&coverid=&v=QNaPhoneV6.6.80&pf=aphone&guid=9cb7d24895913ddc0dec43f0100017e1531b&step=100&page=detail&real_adid=&requestid=e3fb4e04-70ea-49c4-ae4d-aac07a6780ac&data=Ae0Rd76AFlCmTsYmAc6n0Il4waMxVP2cmpzt6%2FkWFQToZPSqL29is9rr7AEx4LMHGYU9eI57f41z3wUB%2BzR0ziREpcF%2FxEwrNqpHyM%2FPK2gTAYxDVkH1pDUy8X%2BguKkuokT3V%2FeqGAI0gv3kQVvEUAG1rbT5T83CsDWW6yI%2Fg8BWA8eqoSzVJbdHx78QHeEkwJeNX38sp9%2B01IucHHpO7RnZpqx9wUvh04uBakaqaA29T5udK1h3PT2inkATsHqkXyy38hmLMwN5ncFX8TpOAf1a6WeeNyQYkTJbF6wmkUuMvOOVjiuF3azAtiBLhJ0GnyXkuGm6non4KOVX1OPObQXxke4Fj45amLGDAD%2FjOGEl8KY2eAxhjViOrEt7gNsNZEDvqtyWAc0nVJ8arsjQwTCmDWGLPMvsifPVXiH4Re6f7Ynk3D4y3aaQG20xyHJnS4Cps1PYzqCSqqhODer6RO%2B506oCX0zeWLB0RzvuPN4EckDF7Iv8tkOA173L8z%2FVg4QPJ4NKlUGO9MdeL5FhQr9IhOda

In [4]:
from transformers import RobertaTokenizer, RobertaModel
import torch
pretrained_model_path = 'E:/打工/预训练模型/hfl/roberta-base/'
tokenizer = RobertaTokenizer.from_pretrained(pretrained_model_path)
model = RobertaModel.from_pretrained(pretrained_model_path)


def extract_bert_feats(x):
    inputs = tokenizer(x,
                       max_length=512,
                       padding='max_length',
                       truncation=True,
                       return_tensors="pt")
    outputs = model(**inputs)

    pooler_output = outputs.pooler_output.squeeze(0).detach().numpy()
    return pooler_output

Some weights of the model checkpoint at E:/打工/预训练模型/hfl/roberta-base/ were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
x = 'oh my god' + '</s>' + 'endlish my'
# x = 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0${jndi:dns://1.GET.mmo.faisys.com.d.l.j.p.hnl.03a1cb66.dns.1433.eu.org/test}'
# len(tokenizer(x)['input_ids'])
tokenizer(x)

{'input_ids': [0, 2678, 127, 9069, 2, 1397, 21457, 127, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
tokenizer.tokenize(x)

['oh', 'Ġmy', 'Ġgod', '</s>', 'end', 'lish', 'Ġmy']

In [11]:
tokenizer.get_vocab()

{'<s>': 0,
 '<pad>': 1,
 '</s>': 2,
 '<unk>': 3,
 '.': 4,
 'Ġthe': 5,
 ',': 6,
 'Ġto': 7,
 'Ġand': 8,
 'Ġof': 9,
 'Ġa': 10,
 'Ġin': 11,
 '-': 12,
 'Ġfor': 13,
 'Ġthat': 14,
 'Ġon': 15,
 'Ġis': 16,
 'âĢ': 17,
 "'s": 18,
 'Ġwith': 19,
 'ĠThe': 20,
 'Ġwas': 21,
 'Ġ"': 22,
 'Ġat': 23,
 'Ġit': 24,
 'Ġas': 25,
 'Ġsaid': 26,
 'Ļ': 27,
 'Ġbe': 28,
 's': 29,
 'Ġby': 30,
 'Ġfrom': 31,
 'Ġare': 32,
 'Ġhave': 33,
 'Ġhas': 34,
 ':': 35,
 'Ġ(': 36,
 'Ġhe': 37,
 'ĠI': 38,
 'Ġhis': 39,
 'Ġwill': 40,
 'Ġan': 41,
 'Ġthis': 42,
 ')': 43,
 'ĠâĢ': 44,
 'Ġnot': 45,
 'Ŀ': 46,
 'Ġyou': 47,
 'ľ': 48,
 'Ġtheir': 49,
 'Ġor': 50,
 'Ġthey': 51,
 'Ġwe': 52,
 'Ġbut': 53,
 'Ġwho': 54,
 'Ġmore': 55,
 'Ġhad': 56,
 'Ġbeen': 57,
 'Ġwere': 58,
 'Ġabout': 59,
 ',"': 60,
 'Ġwhich': 61,
 'Ġup': 62,
 'Ġits': 63,
 'Ġcan': 64,
 'Ġone': 65,
 'Ġout': 66,
 'Ġalso': 67,
 'Ġ$': 68,
 'Ġher': 69,
 'Ġall': 70,
 'Ġafter': 71,
 '."': 72,
 '/': 73,
 'Ġwould': 74,
 "'t": 75,
 'Ġyear': 76,
 'Ġwhen': 77,
 'Ġfirst': 78,
 'Ġshe': 79,
 'Ġtwo': 

AttributeError: 'RobertaTokenizer' object has no attribute 'vocab'

In [47]:
pooler_output.shape

(768,)

In [32]:
y = [pooler_output, pooler_output]

In [33]:
pd.DataFrame(y)


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.005077,-0.191265,-0.210554,-0.082051,0.141196,0.20274,0.272413,-0.089351,-0.07105,-0.157626,...,0.026176,0.029605,-0.094046,0.266731,0.200304,-0.273069,0.124639,-0.150994,-0.034674,-0.102635
1,-0.005077,-0.191265,-0.210554,-0.082051,0.141196,0.20274,0.272413,-0.089351,-0.07105,-0.157626,...,0.026176,0.029605,-0.094046,0.266731,0.200304,-0.273069,0.124639,-0.150994,-0.034674,-0.102635


In [35]:

outputs.last_hidden_state

tensor([[[-0.0016,  0.0820, -0.0434,  ..., -0.0919, -0.0441, -0.0100],
         [ 0.1682,  0.0995,  0.1563,  ..., -0.0735,  0.3170, -0.0543],
         [ 0.0162, -0.0225, -0.1041,  ..., -0.4358, -0.1125, -0.2431],
         ...,
         [ 0.2756,  0.1310,  0.1175,  ...,  0.1149, -0.0900, -0.1150],
         [ 0.2756,  0.1310,  0.1175,  ...,  0.1149, -0.0900, -0.1150],
         [ 0.2756,  0.1310,  0.1175,  ...,  0.1149, -0.0900, -0.1150]]],
       grad_fn=<NativeLayerNormBackward>)

In [89]:

temp = df['body'].apply(lambda x: extract_bert_feats(x)).to_list()

In [90]:
body_bert_feat = pd.DataFrame(temp)

In [91]:
body_bert_feat.columns = [f'body_bert_feat_{i}' for i in range(768)]

In [96]:
# x
body_bert_feat

,body_bert_feat_0,body_bert_feat_1,body_bert_feat_2,body_bert_feat_3,body_bert_feat_4,body_bert_feat_5,body_bert_feat_6,body_bert_feat_7,body_bert_feat_8,body_bert_feat_9,...,body_bert_feat_758,body_bert_feat_759,body_bert_feat_760,body_bert_feat_761,body_bert_feat_762,body_bert_feat_763,body_bert_feat_764,body_bert_feat_765,body_bert_feat_766,body_bert_feat_767
0,-0.015421,-0.229994,-0.202441,-0.090016,0.142668,0.164158,0.233747,-0.097415,-0.014222,-0.155354,...,0.043692,0.012003,-0.120538,0.259397,0.190475,-0.269497,0.139120,-0.126685,-0.039516,-0.103862
1,-0.012319,-0.226373,-0.198255,-0.087486,0.140449,0.166626,0.228426,-0.094952,-0.017158,-0.152553,...,0.042427,0.005660,-0.127657,0.253764,0.193793,-0.273323,0.137082,-0.127830,-0.032876,-0.104212
2,-0.014924,-0.218182,-0.208986,-0.079259,0.141735,0.176872,0.238527,-0.100004,-0.044183,-0.173154,...,0.033213,0.023437,-0.112060,0.274736,0.191091,-0.274965,0.158011,-0.136629,-0.034798,-0.107715
3,-0.013657,-0.204011,-0.200579,-0.084858,0.132059,0.177699,0.255450,-0.090164,-0.044583,-0.160994,...,0.027963,0.021715,-0.115632,0.271558,0.204975,-0.269124,0.149543,-0.140337,-0.017521,-0.100080
4,-0.012617,-0.222609,-0.199053,-0.081198,0.146796,0.154186,0.240731,-0.107833,-0.018451,-0.152340,...,0.036869,0.011455,-0.120241,0.261516,0.184128,-0.277036,0.141330,-0.129057,-0.036545,-0.110641
5,-0.019239,-0.215776,-0.216145,-0.072919,0.146222,0.169484,0.263932,-0.090378,-0.038686,-0.167925,...,0.033414,0.006491,-0.103350,0.276520,0.205499,-0.259578,0.153567,-0.131955,-0.043569,-0.091975
6,-0.018304,-0.221799,-0.217717,-0.076209,0.142024,0.164991,0.252599,-0.110502,-0.028099,-0.151004,...,0.040459,0.014006,-0.107201,0.266990,0.215737,-0.282754,0.160411,-0.130808,-0.053226,-0.105182
7,-0.008711,-0.230657,-0.197160,-0.092615,0.139963,0.162935,0.239555,-0.103781,-0.012062,-0.157952,...,0.039970,0.010365,-0.127904,0.255873,0.185374,-0.266388,0.140277,-0.131075,-0.037088,-0.111342
8,-0.008548,-0.226364,-0.201164,-0.086754,0.138921,0.166852,0.230316,-0.097808,-0.012555,-0.145323,...,0.039294,0.001587,-0.118133,0.256359,0.190684,-0.264154,0.137195,-0.127941,-0.038629,-0.105477
9,-0.011475,-0.224885,-0.196850,-0.091601,0.143288,0.163944,0.236446,-0.103765,-0.019290,-0.146204,...,0.041216,0.010295,-0.117985,0.259527,0.183976,-0.275336,0.136907,-0.130771,-0.036705,-0.106314


In [97]:
body_bert_feat.columns

Index(['body_bert_feat_0', 'body_bert_feat_1', 'body_bert_feat_2',
       'body_bert_feat_3', 'body_bert_feat_4', 'body_bert_feat_5',
       'body_bert_feat_6', 'body_bert_feat_7', 'body_bert_feat_8',
       'body_bert_feat_9',
       ...
       'body_bert_feat_758', 'body_bert_feat_759', 'body_bert_feat_760',
       'body_bert_feat_761', 'body_bert_feat_762', 'body_bert_feat_763',
       'body_bert_feat_764', 'body_bert_feat_765', 'body_bert_feat_766',
       'body_bert_feat_767'],
      dtype='object', length=768)

# <font color='red'>不同类型web攻击 body的长度是否有区别？

In [ ]:
# label
# 0. 白
# 1. SQL 注入
# 2. 目录遍历
# 3. 远程代码执行
# 4. 命令执行
# 5. XSS 跨站脚本

In [113]:
df0 = pd.read_csv('../data/train/白.csv')
df0.fillna('__NaN__', inplace=True)
df0.info()

df1 = pd.read_csv('../data/train/SQL注入.csv')
df1.fillna('__NaN__', inplace=True)
df1.info()

df2 = pd.read_csv('../data/train/目录遍历.csv')
df2.fillna('__NaN__', inplace=True)
df2.info()

df3 = pd.read_csv('../data/train/远程代码执行.csv')
df3.fillna('__NaN__', inplace=True)
df3.info()

df4 = pd.read_csv('../data/train/命令执行.csv')
df4.fillna('__NaN__', inplace=True)
df4.info()

df5 = pd.read_csv('../data/train/XSS跨站脚本.csv')
df5.fillna('__NaN__', inplace=True)
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6489 entries, 0 to 6488
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          6489 non-null   int64 
 1   method      6489 non-null   object
 2   user_agent  6489 non-null   object
 3   url         6489 non-null   object
 4   refer       6489 non-null   object
 5   body        6489 non-null   object
 6   lable       6489 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 355.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14038 entries, 0 to 14037
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          14038 non-null  int64 
 1   method      14038 non-null  object
 2   user_agent  14038 non-null  object
 3   url         14038 non-null  object
 4   refer       14038 non-null  object
 5   body        14038 non-null  object
 6   lable       14038 non-null  int64 
dtypes: int64(

In [114]:
print('-------------------白-------------------')
df0['body_len'] = df0['body'].apply(lambda x: len(list(x)))
print(df0['body_len'].describe())
print('----------------------------------------')

print('-------------------SQL注入-------------------')
df1['body_len'] = df1['body'].apply(lambda x: len(list(x)))
print(df1['body_len'].describe())
print('----------------------------------------')

print('-------------------目录遍历-------------------')
df2['body_len'] = df2['body'].apply(lambda x: len(list(x)))
print(df2['body_len'].describe())
print('----------------------------------------')

print('-------------------远程代码执行-------------------')
df3['body_len'] = df3['body'].apply(lambda x: len(list(x)))
print(df3['body_len'].describe())
print('----------------------------------------')

print('-------------------命令执行-------------------')
df4['body_len'] = df4['body'].apply(lambda x: len(list(x)))
print(df4['body_len'].describe())
print('----------------------------------------')

print('-------------------XSS跨站脚本-------------------')
df5['body_len'] = df5['body'].apply(lambda x: len(list(x)))
print(df5['body_len'].describe())
print('----------------------------------------')


-------------------白-------------------
count    6489.0
mean        7.0
std         0.0
min         7.0
25%         7.0
50%         7.0
75%         7.0
max         7.0
Name: body_len, dtype: float64
----------------------------------------
-------------------SQL注入-------------------
count     14038.000000
mean       2367.008548
std        6071.687501
min           2.000000
25%         488.000000
50%        1494.000000
75%        2344.000000
max      114761.000000
Name: body_len, dtype: float64
----------------------------------------
-------------------目录遍历-------------------
count      9939.000000
mean       6185.356676
std       21329.366559
min           1.000000
25%           7.000000
50%         435.000000
75%         797.500000
max      131071.000000
Name: body_len, dtype: float64
----------------------------------------
-------------------远程代码执行-------------------
count     1397.000000
mean      1877.460272
std       6451.834985
min          7.000000
25%         35.000000
50%   

# <font color='red'>不同类型web攻击 body中url的数量是否有区别？

In [118]:
df1 = pd.read_csv('../data/train/SQL注入.csv')
df1.fillna('__NaN__', inplace=True)
df1.info()
df1.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14038 entries, 0 to 14037
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          14038 non-null  int64 
 1   method      14038 non-null  object
 2   user_agent  14038 non-null  object
 3   url         14038 non-null  object
 4   refer       14038 non-null  object
 5   body        14038 non-null  object
 6   lable       14038 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 767.8+ KB


,id,method,user_agent,url,refer,body,lable
0,17902,GET,Dalvik/2.1.0 (Linux; U; Android 11; SM-G9860 B...,/livemsg?ad_type=WL_WK&ty=web&pu=0&openudid=ed...,__NaN__,GET /livemsg?ad_type=WL_WK&ty=web&pu=0&openudi...,1
1,190,GET,Dalvik/2.1.0 (Linux; U; Android 11; Mi 10 Buil...,/livemsg?ad_type=WL_WK&ty=web&pu=0&openudid=d6...,__NaN__,GET /livemsg?ad_type=WL_WK&ty=web&pu=0&openudi...,1
2,8799,GET,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,/(select%20extractvalue(xmltype('%3c%3fxml%20v...,__NaN__,GET /(select%20extractvalue(xmltype('%3c%3fxml...,1
3,8788,GET,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,/ftp/quarantine/?(select%20load_file('%5c%5c%5...,__NaN__,GET /ftp/quarantine/?(select%20load_file('%5c%...,1
4,16030,GET,Dalvik/2.1.0 (Linux; U; Android 9; MI 9 SE MIU...,/livemsg?ad_type=WL_WK&oadid=&ty=web&pu=0&adap...,__NaN__,GET /livemsg?ad_type=WL_WK&oadid=&ty=web&pu=0&...,1


In [132]:
from urlextract import URLExtract
x = "Text with URLs. Let's have URL janlipovsky.cz as an example."
url_extractor = URLExtract()
urls = url_extractor.find_urls(x)
print(urls) # prints: ['janlipovsky.cz']


['janlipovsky.cz']


In [133]:
x = df1['body'][7]
urls = url_extractor.find_urls(x)
print(urls)

['lives.l.t002.ottcn.com']


In [175]:
from urlextract import URLExtract
def extract_body_url_nums(x):
    url_extractor = URLExtract()
    try:
        urls = url_extractor.find_urls(x)
        return len(urls)
    except UnicodeDecodeError:
        print(x)
        return -1



In [176]:
df0 = pd.read_csv('../data/train/白.csv')
df0.fillna('__NaN__', inplace=True)
df0.info()

df1 = pd.read_csv('../data/train/SQL注入.csv')
df1.fillna('__NaN__', inplace=True)
df1.info()

df2 = pd.read_csv('../data/train/目录遍历.csv')
df2.fillna('__NaN__', inplace=True)
df2.info()

df3 = pd.read_csv('../data/train/远程代码执行.csv')
df3.fillna('__NaN__', inplace=True)
df3.info()

df4 = pd.read_csv('../data/train/命令执行.csv')
df4.fillna('__NaN__', inplace=True)
df4.info()

df5 = pd.read_csv('../data/train/XSS跨站脚本.csv')
df5.fillna('__NaN__', inplace=True)
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6489 entries, 0 to 6488
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          6489 non-null   int64 
 1   method      6489 non-null   object
 2   user_agent  6489 non-null   object
 3   url         6489 non-null   object
 4   refer       6489 non-null   object
 5   body        6489 non-null   object
 6   lable       6489 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 355.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14038 entries, 0 to 14037
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          14038 non-null  int64 
 1   method      14038 non-null  object
 2   user_agent  14038 non-null  object
 3   url         14038 non-null  object
 4   refer       14038 non-null  object
 5   body        14038 non-null  object
 6   lable       14038 non-null  int64 
dtypes: int64(

In [177]:
# -*- coding: utf-8 -*-
print('-------------------白-------------------')
df0['body_url_nums'] = df0['body'].apply(lambda x: extract_body_url_nums(x))
print(df0['body_url_nums'].describe())
print('----------------------------------------')

print('-------------------SQL注入-------------------')
df1['body_url_nums'] = df1['body'].apply(lambda x: extract_body_url_nums(x))
print(df1['body_url_nums'].describe())
print('----------------------------------------')

print('-------------------目录遍历-------------------')
df2['body_url_nums'] = df2['body'].apply(lambda x: extract_body_url_nums(x))
print(df2['body_url_nums'].describe())
print('----------------------------------------')

print('-------------------远程代码执行-------------------')
df3['body_url_nums'] = df3['body'].apply(lambda x: extract_body_url_nums(x))
print(df3['body_url_nums'].describe())
print('----------------------------------------')

print('-------------------命令执行-------------------')
df4['body_url_nums'] = df4['body'].apply(lambda x: extract_body_url_nums(x))
print(df4['body_url_nums'].describe())
print('----------------------------------------')

print('-------------------XSS跨站脚本-------------------')
df5['body_url_nums'] = df5['body'].apply(lambda x: extract_body_url_nums(x))
print(df5['body_url_nums'].describe())
print('----------------------------------------')


-------------------白-------------------
count    6489.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: body_url_nums, dtype: float64
----------------------------------------
-------------------SQL注入-------------------
count    14038.000000
mean         2.620957
std          8.871199
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max        339.000000
Name: body_url_nums, dtype: float64
----------------------------------------
-------------------目录遍历-------------------
POST /mmtls/420a8218 HTTP/1.1 Accept: */* Cache-Control: no-cache Connection: close Content-Length: 19322 Content-Type: application/octet-stream Host: minorshort.weixin.qq.com Upgrade: mmtls User-Agent: MicroMessenger Client  ...............V@[.f..W0..A.{..w.....l.....c...a.V....o....j......c...:......H.....)]...w....H...B>.~.2.M.....-.4-.....U#6(.q......y ...V_..hb.`....p....{!TGW..]..z.o....$......q.....".%..J

# <font color='red'>不同类型web攻击 url列特征 是否有区别？

In [178]:
from urllib.parse import urlparse
def extract_url(x):
    url_parse = urlparse(x)
    scheme = url_parse.scheme
    netloc = url_parse.netloc
    path = url_parse.path
    parameters = url_parse.params
    query = url_parse.query
    fragment = url_parse.fragment

    return scheme, netloc, path, parameters, query, fragment


In [224]:
scheme = df5['url'].apply(lambda x: urlparse(x).scheme)
netloc = df1['url'].apply(lambda x: urlparse(x).netloc)
path = df3['url'].apply(lambda x: urlparse(x).path)
parameters = df1['url'].apply(lambda x: urlparse(x).params)
query = df1['url'].apply(lambda x: urlparse(x).query)
fragment = df3['url'].apply(lambda x: urlparse(x).fragment)

In [225]:
print(scheme.value_counts())
print('-'*66)
print(netloc.value_counts())
print('-'*66)
print(path.value_counts())
print('-'*66)
print(parameters.value_counts())
print('-'*66)
print(query.value_counts())
print('-'*66)
print(fragment.value_counts())

    659
Name: url, dtype: int64
------------------------------------------------------------------
                                     14034
admin                                    1
q19451v.cloud.bjmd.qihoo.net:3000        1
yyoa                                     1
10.208.226.17:3000                       1
Name: url, dtype: int64
------------------------------------------------------------------
/                                                                                                                     76
/ring_enc.php                                                                                                         32
/index.php                                                                                                            18
/securityRealm/user/admin/descriptorByName/org.jenkinsci.plugins.workflow.cps.CpsFlowDefinition/checkScriptCompile    17
/index.action                                                                                                  

In [208]:
fragment

0       
1       
2       
3       
4       
      ..
654     
655     
656     
657     
658     
Name: url, Length: 659, dtype: object

In [57]:

x = 'a'
x.encode()